In [ ]:
!pip install snowflake-connector-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0


In [ ]:
import snowflake.connector
import pandas as pd
import numpy as np
from datetime import datetime
import os, sys, json

In [ ]:
# Configuração dos parâmetros de conexão
account = 'vajdfge-ir30409'


user = 'LLUKAZ'
password = 'Luc421996'
warehouse = 'COMPUTE_WH'
database = 'sf100'
schema = 'PUBLIC'

# Conecta ao Snowflake
con = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)


In [ ]:
queries =  [
        "select sum(TO_DOUBLE(lo_extendedprice*lo_discount)) as revenue from lineorder, dwdate where d_yearmonthnum = 199401 and lo_orderdate = d_datekey and lo_discount between 4 and 6 and lo_quantity between 26 and 35;",
        "select sum(TO_DOUBLE(lo_extendedprice*lo_discount)) as revenue from lineorder, dwdate where d_year = 1993 and lo_orderdate = d_datekey and lo_discount between 1 and 3 and lo_quantity < 25;",
        "select sum(TO_DOUBLE(lo_extendedprice*lo_discount)) as revenue from lineorder, dwdate where d_weeknuminyear = 6 and lo_orderdate = d_datekey and d_year = 1994 and lo_discount between 5 and 7 and lo_quantity between 26 and 35;",
        "select sum(TO_DOUBLE(lo_revenue)), d_year, p_brand from lineorder, dwdate, part, supplier where p_category = 'MFGR#12' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey and s_region = 'AMERICA' group by d_year, p_brand order by d_year, p_brand;",
        "select sum(TO_DOUBLE(lo_revenue)), d_year, p_brand from lineorder, dwdate, part, supplier where p_brand BETWEEN 'MFGR#2221' and  'MFGR#2228' and s_region = 'ASIA' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey group by d_year, p_brand order by d_year, p_brand;",
        "select sum(TO_DOUBLE(lo_revenue)), d_year, p_brand from lineorder, dwdate, part, supplier where p_brand = 'MFGR#2221' and lo_orderdate = d_datekey and lo_partkey = p_partkey and lo_suppkey = s_suppkey and s_region = 'EUROPE' group by d_year, p_brand order by d_year, p_brand;",
        "select c_nation, s_nation, d_year, sum(TO_DOUBLE(lo_revenue)) as revenue from customer, lineorder, supplier, dwdate where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and c_region = 'ASIA' and s_region = 'ASIA' and d_year >= 1992 and d_year <= 1997 group by c_nation, s_nation, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum(TO_DOUBLE(lo_revenue)) as revenue from customer, lineorder, supplier, dwdate where c_nation = 'UNITED STATES' and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and s_nation = 'UNITED STATES' and d_year >= 1992 and d_year <= 1997 group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum(TO_DOUBLE(lo_revenue)) as revenue from customer, lineorder, supplier, dwdate where d_year >= 1992 and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') and d_year <= 1997 group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select c_city, s_city, d_year, sum(TO_DOUBLE(lo_revenue)) as revenue from customer, lineorder, supplier, dwdate where d_yearmonth = 'Dec1997' and lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_orderdate = d_datekey and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') group by c_city, s_city, d_year order by d_year asc, revenue desc;",
        "select d_year, c_nation, sum(TO_DOUBLE(lo_revenue - lo_supplycost)) as profit from dwdate, customer, supplier, part, lineorder where lo_custkey = c_custkey  and lo_suppkey = s_suppkey  and lo_partkey = p_partkey  and lo_orderdate = d_datekey  and c_region = 'AMERICA'  and s_region = 'AMERICA'  and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_year, c_nation order by d_year, c_nation;",
        "select d_year, s_nation, p_category, sum(TO_DOUBLE(lo_revenue - lo_supplycost)) as profit from dwdate, customer, supplier, part, lineorder where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_partkey = p_partkey and lo_orderdate = d_datekey and c_region = 'AMERICA' and s_region = 'AMERICA' and (d_year = 1997 or d_year = 1998) and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_year, s_nation, p_category order by d_year, s_nation, p_category;",
        "select d_year, s_city, p_brand, sum(TO_DOUBLE(lo_revenue - lo_supplycost)) as profit from dwdate, customer, supplier, part, lineorder where lo_custkey = c_custkey and lo_suppkey = s_suppkey and lo_partkey = p_partkey and lo_orderdate = d_datekey and c_region = 'AMERICA' and s_nation = 'UNITED STATES' and (d_year = 1997 or d_year = 1998) and p_category = 'MFGR#14' group by d_year, s_city, p_brand order by d_year, s_city, p_brand;"
        ]


In [ ]:
queries_flat_table = [
    "select sum(lo_extendedprice*lo_discount) as revenue from lineorder_obt where d_order_yearmonthnum = 199401 and lo_discount between 4 and 6 and lo_quantity between 26 and 35;",
    "select sum(lo_extendedprice*lo_discount) as revenue from lineorder_obt where d_order_year = 1993 and lo_discount between 1 and 3 and lo_quantity < 25;",
    "select sum(lo_extendedprice*lo_discount) as revenue from lineorder_obt where d_order_weeknuminyear = 6 and d_order_year = 1994 and lo_discount between 5 and 7 and lo_quantity between 26 and 35;",
    "select sum(lo_revenue), d_order_year, p_brand from lineorder_obt where p_category = 'MFGR#12' and s_region = 'AMERICA' group by d_order_year, p_brand order by d_order_year, p_brand;",
    "select sum(lo_revenue), d_order_year, p_brand from lineorder_obt where p_brand between 'MFGR#2221' and 'MFGR#2228' and s_region = 'ASIA' group by d_order_year, p_brand order by d_order_year, p_brand;",
    "select sum(lo_revenue), d_order_year, p_brand from lineorder_obt where p_brand = 'MFGR#2221' and s_region = 'EUROPE' group by d_order_year, p_brand order by d_order_year, p_brand;",
    "select c_nation, s_nation, d_order_year, sum(lo_revenue) as revenue from lineorder_obt where c_region = 'ASIA' and s_region = 'ASIA' and d_order_year >= 1992 and d_order_year <= 1997 group by c_nation, s_nation, d_order_year order by d_order_year asc, revenue desc;",
    "select c_city, s_city, d_order_year, sum(lo_revenue) as revenue from lineorder_obt where c_nation = 'UNITED STATES' and s_nation = 'UNITED STATES' and d_order_year >= 1992 and d_order_year <= 1997 group by c_city, s_city, d_order_year order by d_order_year asc, revenue desc;",
    "select c_city, s_city, d_order_year, sum(lo_revenue) as revenue from lineorder_obt where d_order_year >= 1992 and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') and d_order_year <= 1997 group by c_city, s_city, d_order_year order by d_order_year asc, revenue desc;",
    "select c_city, s_city, d_order_year, sum(lo_revenue) as revenue from lineorder_obt where d_order_yearmonth = 'Dec1997' and (c_city='UNITED KI1' or c_city='UNITED KI5') and (s_city='UNITED KI1' or s_city='UNITED KI5') group by c_city, s_city, d_order_year order by d_order_year asc, revenue desc;",
    "select d_order_year, c_nation, sum(lo_revenue - lo_supplycost) as profit from lineorder_obt where c_region = 'AMERICA'  and s_region = 'AMERICA'  and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_order_year, c_nation order by d_order_year, c_nation;",
    "select d_order_year, s_nation, p_category, sum(lo_revenue - lo_supplycost) as profit from lineorder_obt where c_region = 'AMERICA' and s_region = 'AMERICA' and (d_order_year = 1997 or d_order_year = 1998) and (p_mfgr = 'MFGR#1' or p_mfgr = 'MFGR#2') group by d_order_year, s_nation, p_category order by d_order_year, s_nation, p_category;",
    "select d_order_year, s_city, p_brand, sum(lo_revenue - lo_supplycost) as profit from lineorder_obt where c_region = 'AMERICA' and s_nation = 'UNITED STATES' and (d_order_year = 1997 or d_order_year = 1998) and p_category = 'MFGR#14' group by d_order_year, s_city, p_brand order by d_order_year, s_city, p_brand;"
]

In [ ]:
# Cria um cursor
cursor = con.cursor()

query = """
select /*+ NO_CACHE */ sum(TO_DOUBLE(lo_extendedprice*lo_discount)) as revenue from lineorder, dwdate where d_yearmonthnum = 199401 and lo_orderdate = d_datekey and lo_discount between 4 and 6 and lo_quantity between 26 and 35;
"""

# Executa uma consulta
cursor.execute(query)

# Obtém o ID da última consulta
query_id = cursor.sfqid

# Executa uma consulta para recuperar informações do Query Profile
exec = cursor.execute(f"""
select * from table(GET_QUERY_OPERATOR_STATS('{query_id}'));
""")

# Recupera os resultados
query_history_results = exec.fetchall()

# Exibe informações do Query Profile
if query_history_results:
    query_profile_json = query_history_results[0][1]  # Ajuste o índice conforme necessário
    print(f"Query Profile JSON:\n{query_profile_json}")

# Fecha a conexão
cursor.close()
con.close()

Query Profile JSON:
1


In [ ]:
for i in range(len(queries_flat_table)):
  arr_elapsed_time = []
  arr_cpu_time = []
  arr_bytes_scanned = []

  arr_local_disk_io = []
  arr_overall_percentage = []
  arr_remote_disk_io = []
  arr_output_rows = []
  for _ in range(1,31):
    # Cria um cursor
    cursor = con.cursor()



    cursor.execute("alter session set USE_CACHED_RESULT = FALSE;")

    start_time = datetime.utcnow()
    consulta = f"""
    /*+ NO_CACHED_RESULTS */
    /*+ NO_QUERY_RESULT_REUSE */
    {queries_flat_table[i]}
    """



    # Executa uma consulta
    cursor.execute(consulta)

    end_time = datetime.utcnow()

    # Obtém o ID da última consulta
    query_id = cursor.sfqid


    # Calcula o tempo decorrido
    elapsed_time = (end_time - start_time).total_seconds()

    arr_elapsed_time.append(elapsed_time*1000)

    #print(f"Elapsed Time (seconds): {elapsed_time}")



    query = f"select * from table(GET_QUERY_OPERATOR_STATS('{query_id}'))"
    results = cursor.execute(query).fetchall()
    for row in results:
      nodeId = str(row[2])
      parentId = row[3]
      step = str(row[4])
      linha_6 = str(row[6])
      linha_5 = str(row[5])



    dictionary = json.loads(str(row[6]))
    objeto = json.loads(str(row[5]))

    #arr_cpu_time.append(round(float(dictionary['processing'])*1000, 3))
    arr_bytes_scanned.append(objeto['io']['bytes_scanned'])
    #arr_local_disk_io.append(dictionary['local_disk_io'])
    arr_overall_percentage.append(dictionary['overall_percentage'])
    #arr_remote_disk_io.append(dictionary['remote_disk_io'])
    arr_output_rows.append(objeto['output_rows'])

    df = pd.DataFrame()

    lista_de_tuplas = list(zip(arr_elapsed_time, arr_bytes_scanned, arr_overall_percentage, arr_output_rows))
    df = pd.DataFrame(
      lista_de_tuplas,
      columns=['elapsed_time_ms', 'bytes_scanned', 'overall_percentage', 'output_rows']
    )

    df.to_csv('sf100_small_4nos_flat_query_'+str(i)+'.csv')


https://github.com/cristiscu/custom-query-profiler/blob/master/custom-query-profiler.py